# Building a Streamlit application to act as our AI Assistant

You need to run this **Streamlit app separately** from your Jupyter notebook. **Streamlit** runs as a web app and doesn't natively integrate within a Jupyter notebook like standard Python scripts. Here's how you can run it:

### **Steps to Run the Streamlit App**
1. **Save the Code**  
   - Save the provided code in a Python script, e.g., `app.py`.
  
```python
import streamlit as st
import os
import re
from dotenv import load_dotenv
import google.generativeai as genai
from openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from anthropic import Anthropic
from langchain_core.messages import HumanMessage  

# Load environment variables
load_dotenv()

def get_env_var(var: str):
    value = os.getenv(var)
    if value is None:
        raise ValueError(f"{var} not found in environment variables. Make sure it is set in your .env file.")
    return value

# Load API keys
openai_api_key = get_env_var("OPENAI_COURSE_KEY")  
gemini_api_key = get_env_var("GEMINI_API_KEY")  
anthropic_api_key = get_env_var("ANTHROPIC_API_KEY")  
xai_api_key = get_env_var("XAI_API_KEY")

# Initialize LLMs
gpt4o_chat = ChatOpenAI(model="gpt-4o", temperature=0, openai_api_key=openai_api_key)
gpt4o_mini_chat = ChatOpenAI(model="gpt-4o-mini", temperature=0, openai_api_key=openai_api_key)
claude_chat = ChatAnthropic(model="claude-3-5-sonnet-20241022", temperature=0, anthropic_api_key=anthropic_api_key)
genai.configure(api_key=gemini_api_key)  
gemini_model = genai.GenerativeModel("gemini-2.0-flash")
grok_client = OpenAI(api_key=xai_api_key, base_url="https://api.x.ai/v1")

def query_grok(prompt: str):
    try:
        completion = grok_client.chat.completions.create(
            model="grok-2-latest",
            messages=[
                {"role": "system", "content": "You are Grok, a chatbot inspired by the Hitchhiker's Guide to the Galaxy."},
                {"role": "user", "content": prompt}
            ],
        )
        return completion.choices[0].message.content
    except Exception as e:
        return f"Error querying Grok: {e}"

# Streamlit App Title
st.title("LLM & Agent Interaction App")

# Sidebar Configuration
st.sidebar.header("Configuration")
llm_provider = st.sidebar.selectbox("Choose LLM Provider", ["GPT-4o", "GPT-4o-mini", "Claude-3.5-Sonnet", "Gemini-2.0-Flash", "Grok-2-Latest"])
user_input = st.text_area("Enter your prompt:")

if st.button("Submit Query"):
    if not user_input:
        st.warning("Please enter a prompt before submitting.")
    else:
        try:
            if llm_provider == "GPT-4o":
                response = gpt4o_chat.invoke([HumanMessage(content=user_input)]).content
            elif llm_provider == "GPT-4o-mini":
                response = gpt4o_mini_chat.invoke([HumanMessage(content=user_input)]).content
            elif llm_provider == "Claude-3.5-Sonnet":
                response = claude_chat.invoke(user_input).content
            elif llm_provider == "Gemini-2.0-Flash":
                response = gemini_model.generate_content(user_input).text
            elif llm_provider == "Grok-2-Latest":
                response = query_grok(user_input)
            else:
                response = "Invalid model selection."
            st.success(response)
        except Exception as e:
            st.error(f"Error: {e}")

```

2. **Install Streamlit et al (if not installed)**
   ```bash

        place below pip installs into your requirements.txt
   
        streamlit
        requests
        python-dotenv
        google-generativeai
        google-genai
        openai
        anthropic
        langchain-openai
        langchain-anthropic
        langchain-core
        langchain-community

        python3 -m venv AI-Assistant
        source AI-Assistant/bin/activate
        pip install -r requirements.txt



   ```

3. **Run the App**
   Navigate to the directory where your `app.py` file is located and run:
   ```bash
   streamlit run app.py
   ```
   This will launch a web interface in your browser.



## Adding more advanced features to our AI Assistant

```python
import streamlit as st
import os
import requests
from dotenv import load_dotenv
import google.generativeai as genai
from openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from anthropic import Anthropic
from langchain_core.messages import HumanMessage  

# Load environment variables
load_dotenv()

def get_env_var(var: str):
    value = os.getenv(var)
    if value is None:
        raise ValueError(f"{var} not found in environment variables. Make sure it is set in your .env file.")
    return value

# Load API keys
openai_api_key = get_env_var("OPENAI_API_COURSE_KEY")  
gemini_api_key = get_env_var("GEMINI_API_KEY")  
anthropic_api_key = get_env_var("ANTHROPIC_API_KEY")  
xai_api_key = get_env_var("XAI_API_KEY")
tavily_api_key = get_env_var("TAVILY_API_KEY")

# Initialize LLMs
gpt4o_chat = ChatOpenAI(model="gpt-4o", temperature=0, openai_api_key=openai_api_key)
gpt4o_mini_chat = ChatOpenAI(model="gpt-4o-mini", temperature=0, openai_api_key=openai_api_key)
claude_chat = ChatAnthropic(model="claude-3-5-sonnet-20241022", temperature=0, anthropic_api_key=anthropic_api_key)
genai.configure(api_key=gemini_api_key)  
gemini_model = genai.GenerativeModel("gemini-2.0-flash")
grok_client = OpenAI(api_key=xai_api_key, base_url="https://api.x.ai/v1")

def query_grok(prompt: str):
    try:
        completion = grok_client.chat.completions.create(
            model="grok-2-latest",
            messages=[
                {"role": "user", "content": prompt}
            ],
        )
        return completion.choices[0].message.content
    except Exception as e:
        return f"Error querying Grok: {e}"

def query_tavily(search_query: str):
    """Perform a web search using Tavily API."""
    try:
        url = "https://api.tavily.com/search"
        headers = {"Authorization": f"Bearer {tavily_api_key}", "Content-Type": "application/json"}
        payload = {"query": search_query}
        response = requests.post(url, json=payload, headers=headers)
        data = response.json()
        return data.get("results", [])
    except Exception as e:
        return [f"Error querying Tavily: {e}"]

def generate_follow_up_queries(prompt: str, response: str):
    """Generate LLM-based follow-up queries based on the original query and response."""
    follow_up_prompt = f"Based on the following user query and response, suggest three relevant follow-up questions:\n\nUser Query: {prompt}\n\nResponse: {response}\n\nFollow-up Questions:"
    try:
        follow_up_text = gpt4o_chat.invoke([HumanMessage(content=follow_up_prompt)]).content
        questions = [q.strip() for q in follow_up_text.split("\n") if q.strip()]
        return questions
    except Exception as e:
        return [f"Error generating follow-up questions: {e}"]

# Streamlit App Title
st.title("AI Assistant")

# Sidebar Configuration
st.sidebar.header("Configuration")
llm_provider = st.sidebar.selectbox("Choose LLM Provider", ["GPT-4o", "GPT-4o-mini", "Claude-3.5-Sonnet", "Gemini-2.0-Flash", "Grok-2-Latest"])
user_persona = st.sidebar.text_input("User Persona", "General User")
system_persona = st.sidebar.text_input("System Persona", "AI Assistant")
response_length = st.sidebar.radio("Response Length", ["Succinct", "Standard", "Thorough"], index=1)
temperature_setting = st.sidebar.radio("Conversation Type (Temperature)", ["Creative", "Balanced", "Precise"], index=1)
num_references = st.sidebar.slider("Number of Referenced Responses", 1, 10, 5)
follow_up_enabled = st.sidebar.checkbox("Enable Follow-up Queries")

# Initialize session state variables
if "process_query" not in st.session_state:
    st.session_state.process_query = False
if "web_search" not in st.session_state:
    st.session_state.web_search = False
if "user_input" not in st.session_state:
    st.session_state.user_input = ""
if "search_results" not in st.session_state:
    st.session_state.search_results = []

# Query Interface at the Top
st.subheader("Enter Your Query")

with st.form(key="query_form"):
    st.text_area("Enter your prompt:", key="user_input")
    col1, col2 = st.columns([0.5, 0.5])
    with col1:
        submit_button = st.form_submit_button("Submit Query")
    with col2:
        web_search_button = st.form_submit_button("Web Search with Tavily")

if submit_button:
    if not st.session_state.user_input:
        st.warning("Please enter a prompt before submitting.")
    else:
        try:
            temperature_values = {"Creative": 0.8, "Balanced": 0.5, "Precise": 0.2}
            temperature = temperature_values[temperature_setting]
            
            if llm_provider == "GPT-4o":
                response = gpt4o_chat.invoke([HumanMessage(content=st.session_state.user_input)], temperature=temperature).content
            elif llm_provider == "GPT-4o-mini":
                response = gpt4o_mini_chat.invoke([HumanMessage(content=st.session_state.user_input)], temperature=temperature).content
            elif llm_provider == "Claude-3.5-Sonnet":
                response = claude_chat.invoke(st.session_state.user_input).content
            elif llm_provider == "Gemini-2.0-Flash":
                response = gemini_model.generate_content(st.session_state.user_input).text
            elif llm_provider == "Grok-2-Latest":
                response = query_grok(st.session_state.user_input)
            else:
                response = "Invalid model selection."
            
            st.subheader("Response")
            st.success(f"{system_persona}: {response}")
            
            if follow_up_enabled:
                st.subheader("Follow-up Questions")
                follow_up_questions = generate_follow_up_queries(st.session_state.user_input, response)
                for question in follow_up_questions:
                    st.write(f"- {question}")
        except Exception as e:
            st.error(f"Error: {e}")

if web_search_button:
    if not st.session_state.user_input:
        st.warning("Please enter a search query before submitting.")
    else:
        search_results = query_tavily(st.session_state.user_input)
        st.subheader("Tavily Search Results:")
        if search_results:
            for idx, result in enumerate(search_results[:5]):
                st.markdown(f"**{idx+1}. [{result['title']}]({result['url']})**")
                st.write(f"{result['content']}")
        else:
            st.write("No relevant search results found.")
```

In [2]:
import os
import requests
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("OPENAI_API_COURSE_KEY")
url = "https://api.openai.com/v1/models"

headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json",
}

response = requests.get(url, headers=headers)
if response.ok:
    models = response.json().get("data", [])
    for model in models:
        print(model["id"])
else:
    print("Error retrieving models:", response.text)



gpt-4.5-preview
gpt-4.5-preview-2025-02-27
gpt-4o-mini-audio-preview-2024-12-17
dall-e-3
dall-e-2
gpt-4o-audio-preview-2024-10-01
gpt-4o-audio-preview
gpt-4o-mini-realtime-preview-2024-12-17
gpt-4o-mini-realtime-preview
o1-mini-2024-09-12
o1-mini
omni-moderation-latest
gpt-4o-mini-audio-preview
omni-moderation-2024-09-26
whisper-1
gpt-4o-realtime-preview-2024-10-01
babbage-002
gpt-4-turbo-preview
chatgpt-4o-latest
tts-1-hd-1106
text-embedding-3-large
gpt-4-0125-preview
gpt-4o-audio-preview-2024-12-17
gpt-4o-2024-05-13
gpt-4
o3-mini-2025-01-31
o3-mini
tts-1-hd
o1-preview
o1-preview-2024-09-12
gpt-3.5-turbo-instruct-0914
tts-1
tts-1-1106
davinci-002
gpt-3.5-turbo-1106
gpt-4-turbo
gpt-3.5-turbo-instruct
o1
gpt-4o-2024-08-06
gpt-3.5-turbo-0125
gpt-4o-realtime-preview-2024-12-17
gpt-3.5-turbo
gpt-4-turbo-2024-04-09
gpt-4o-realtime-preview
gpt-3.5-turbo-16k
gpt-4o-mini-2024-07-18
gpt-4o
gpt-4o-mini
text-embedding-3-small
gpt-4-1106-preview
text-embedding-ada-002
gpt-4-0613
o1-2024-12-17
gpt-

In [3]:
# Import necessary libraries
from dotenv import load_dotenv
import os
import requests

# Load environment variables from your .env file
load_dotenv()

# Retrieve the Google API key from the .env file.
# Ensure your .env file has a line like: GEMINI_API_KEY=your_google_api_key_here
google_api_key = os.getenv("GEMINI_API_KEY")
if not google_api_key:
    raise ValueError("GEMINI_API_KEY not found in the .env file.")

# Construct the URL with the API key as a query parameter
url = f"https://generativelanguage.googleapis.com/v1beta2/models?key={google_api_key}"

# Call the Google API endpoint
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()
    # Extract the list of models (assuming the response contains a "models" key)
    models = data.get("models", [])
    # Print out the list of models
    print("Available Google Models:")
    for model in models:
        print(model.get("name", "No name provided"))
else:
    print("Error retrieving Google models:", response.text)


Available Google Models:
models/chat-bison-001
models/text-bison-001
models/embedding-gecko-001
